
# Tutorial 8 — Metrics (bitstrings, classical info, probabilities, entanglement, states)

This tutorial shows how to use QCOM’s **metrics** utilities to analyze probability data and simulated states.

**What you'll learn**
- Bitstring helpers: reordering and subselecting bits in dictionaries
- Classical information metrics: Shannon entropy, reduced entropy, mutual information, conditional entropy
- Probability diagnostics: cumulative distributions and $N(p)$
- Entanglement metrics: Von Neumann entropy from a state or a Hamiltonian
- State utilities: building density matrices and partial traces

> This notebook assumes you’ve installed/imported `qcom` and that your `qcom.metrics` modules match the versions pasted earlier.


In [1]:

import numpy as np
import qcom as qc

np.set_printoptions(precision=4, suppress=True)



## 1) A small toy probability dictionary

We'll start with a handcrafted distribution over 3-bit outcomes. Keys follow the **MSB ↔ site 0** convention.


In [3]:

probs_3 = {
    "000": 0.10,
    "001": 0.05,
    "010": 0.25,
    "011": 0.10,
    "100": 0.20,
    "101": 0.05,
    "110": 0.20,
    "111": 0.05,
}
print("Sum:", sum(probs_3.values()))
print("First few:")
qc.print_most_probable_data(probs_3, n=3)


Sum: 1.0
First few:
Top 3 Most probable bit strings:
1.  Bit string: 010, Probability: 0.25000000
2.  Bit string: 100, Probability: 0.20000000
3.  Bit string: 110, Probability: 0.20000000



## 2) Bitstring utilities

### 2.1 `order_dict`
Sort entries by the **integer value** of the bitstring key (useful for stable plots or tables).


In [4]:

ordered = qc.metrics.bitstrings.order_dict(probs_3)
list(ordered.items())[:6]


[('000', 0.1),
 ('001', 0.05),
 ('010', 0.25),
 ('011', 0.1),
 ('100', 0.2),
 ('101', 0.05)]


### 2.2 `part_dict`
Select a subset of bit **positions** (with MSB at index 0).  
For example, pick positions `[0, 2]` (MSB and LSB) and aggregate values.


In [5]:

subset_0_2 = qc.metrics.bitstrings.part_dict(probs_3, indices=[0, 2])
ordered_subset = qc.metrics.bitstrings.order_dict(subset_0_2)
ordered_subset, sum(ordered_subset.values())


({'00': 0.35, '01': 0.15000000000000002, '10': 0.4, '11': 0.1}, 1.0)


## 3) Classical information metrics

All entropy-like functions accept a `base` parameter (`2` for bits, `np.e` for nats).

### 3.1 Shannon entropy $H(AB)$


In [6]:

H_AB = qc.compute_shannon_entropy(probs_3, total_prob=None, base=2)
H_AB


2.7414460711655217


### 3.2 Reduced entropies $H(A)$ and $H(B)$

Let’s split 3 sites into **A = {0,1}** and **B = {2}** using a configuration vector where `0` marks A and `1` marks B, e.g.:

- configuration = `[0, 0, 1]`  (two sites in A, one site in B)


In [7]:

configuration = [0, 0, 1]  # sites {0,1} in A; site {2} in B
H_A = qc.compute_reduced_shannon_entropy(probs_3, configuration=configuration, target_region=0, base=2)
H_B = qc.compute_reduced_shannon_entropy(probs_3, configuration=configuration, target_region=1, base=2)
H_A, H_B


(1.9406454496153465, 0.8112781244591328)


### 3.3 Mutual information and conditional entropy


In [8]:

I_AB, H_A_mi, H_B_mi, H_AB_mi = qc.compute_mutual_information(probs_3, configuration=configuration, base=2)
H_A_given_B = qc.compute_conditional_entropy(probs_3, configuration=configuration, base=2)
I_AB, H_A_mi, H_B_mi, H_AB_mi, H_A_given_B


(0.010477502908957437,
 1.9406454496153465,
 0.8112781244591328,
 2.7414460711655217,
 1.9301679467063888)


## 4) Probability diagnostics

### 4.1 Cumulative probability at a threshold


In [9]:

threshold = 0.1
cprob = qc.cumulative_probability_at_value(probs_3, threshold)
threshold, cprob


AttributeError: module 'qcom' has no attribute 'cumulative_probability_at_value'


### 4.2 Cumulative distribution curve

- **Unique-probability** mode (`grid=None`) returns a step function.
- **User grid** mode lets you pass any thresholds (linear or log-spaced).


In [10]:

# Unique-probability step function
x_u, y_u = qc.cumulative_distribution(probs_3, grid=None)
x_u, y_u


(array([0.05, 0.1 , 0.2 , 0.25, 1.  ]), array([0.15, 0.35, 0.75, 1.  , 1.  ]))

In [11]:

# Example: log10-spaced grid from min nonzero prob to 1.0
p_min = min(v for v in probs_3.values() if v > 0)
grid = np.logspace(np.log10(p_min), 0.0, num=8, base=10)
x_g, y_g = qc.cumulative_distribution(probs_3, grid=grid)
x_g, y_g


(array([0.05  , 0.0767, 0.1177, 0.1805, 0.277 , 0.4249, 0.6518, 1.    ]),
 array([0.  , 0.15, 0.35, 0.35, 1.  , 1.  , 1.  , 1.  ]))


### 4.3 $N(p)$ diagnostic (bulk)

Compute $N(p)$ across unique probabilities with a log-window (default `log_base=10`).  
Smaller/larger `p_delta` shrink/expand the multiplicative window.


In [14]:

uniq_p, Np = qc.metrics.probabilities.compute_N_of_p_all(probs_3, p_delta=0.2, show_progress=False, log_base=10.0)
list(zip(uniq_p, Np))[:8]


[(0.05, 129.14413380297975),
 (0.1, 43.04804460099324),
 (0.2, 34.97653623830701),
 (0.25, 22.38498319251649)]


## 5) States and entanglement

### 5.1 From a statevector to probabilities


In [15]:

# 2-qubit Bell state |Φ+> = (|00> + |11>) / √2  in MSB↔site0 convention
psi_bell = np.zeros(4, dtype=np.complex128)
psi_bell[0] = 1/np.sqrt(2)
psi_bell[3] = 1/np.sqrt(2)

probs_bell = qc.statevector_to_probabilities(psi_bell, msb_site0=True)
probs_bell, sum(probs_bell.values())


({'00': 0.5, '11': 0.5}, 1.0)


### 5.2 Density matrix and partial trace
Compute the reduced density matrix of the first qubit (keep site 0).


In [17]:

rho_full = qc.metrics.states.create_density_matrix(psi_bell)
# configuration: 1 = keep, 0 = trace out → keep site 0, trace site 1
cfg_keep0 = [1, 0]
rho_A = qc.metrics.states.compute_reduced_density_matrix(rho_full, configuration=cfg_keep0)
rho_full, rho_A


(array([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
        [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
        [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
        [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]]),
 array([[0.5+0.j, 0. +0.j],
        [0. +0.j, 0.5+0.j]]))


### 5.3 Von Neumann entanglement entropy (from state)

For the Bell state, the reduced density matrix is maximally mixed → entropy = 1 bit.


In [18]:

S_bits = qc.von_neumann_entropy_from_state(psi_bell, configuration=cfg_keep0, base=2)
S_nats = qc.von_neumann_entropy_from_state(psi_bell, configuration=cfg_keep0, base=np.e)
S_bits, S_nats


(1.0000000000000002, 0.6931471805599454)


## 6) (Optional) Eigenstate probabilities from a Hamiltonian

If you have a Hamiltonian compatible with QCOM’s static solver (dense array, SciPy sparse, or `LinearOperator`), you can map the eigenstate into computational-basis probabilities without densifying the operator.


In [19]:

# Example: a simple 1-qubit Hamiltonian H = Z = diag(1, -1)
H1 = np.diag([1.0, -1.0])

# Ground state index is 0 by default; for this toy, the solver should return |1> as ground if it orders ascending.
# If your solver orders differently, you can set state_index accordingly.
try:
    probs_eig = qc.get_eigenstate_probabilities(H1, state_index=0, show_progress=False, drop_tol=0.0, msb_site0=True)
    probs_eig
except Exception as e:
    print("This cell requires qcom.solvers.static to be available. If it's not installed, skip this section.\n", e)



---

### Wrap-up
You now have a tour of:
- **Bitstring** tools for reindexing/aggregating dictionaries.
- **Classical** info metrics (entropies, MI, conditional entropy) with configurable log bases.
- **Probability** diagnostics: cumulative distributions and $N(p)$.
- **Entanglement** from states/Hamiltonians, and **state** helpers for $\rho$ and partial traces.

Try swapping in your experimental/simulation data and larger systems.
